In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from datetime import datetime

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import uuid
import os


class GoogleDriveDatabase:
  def __init__(self, drive, DATABASE_GID:str):
    assert isinstance(DATABASE_GID, str)
    self.folders = {}
    self.drive = drive
    folder_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(DATABASE_GID)}).GetList()
    for file in folder_list:
      if file['mimeType'] == "application/vnd.google-apps.folder":
        self.folders[file['title']] = file['id']
    print("{} folders loaded".format(len(self.folders.keys())))
  def upload(self, filename, character, fileType):
    assert isinstance(filename, str)
    assert isinstance(character, str)
    assert isinstance(fileType, str)
    FILETYPE_MIME_MAP = {
        "jpeg" : "image/jpeg",
        "json" : "application/json",
        "zip" : "application/zip"
    }
    assert fileType in FILETYPE_MIME_MAP.keys(), "fileType must be one of the following: {}".format(fileType)
    assert os.path.isfile(filename), "{} does not exist as a file".format(filename)
    assert self.checkFolder(character), "{} is not a valid character. Pick from list: \n{}".format(character, tuple(self.folders.keys()))
    file = self.drive.CreateFile({
        "title" :  os.path.split(filename)[1],
        "mimeType" : FILETYPE_MIME_MAP[fileType],
        "parents" : [{"id" : self.folders[character]}]
    })
    file.SetContentFile(filename)
    file.Upload()
    os.remove(filename)
    print("uploaded and deleted {}".format(filename))

  @staticmethod
  def FILE_EXTENSIONS() -> list:
    return [".jpg", ".jpeg", ".png", ".zip", ".json"]

  def getFiles(self, character) -> list:
    return [
      x for x in self.drive.ListFile({'q': "'{}' in parents and trashed=false".format(self.folders[character])}).GetList()
      if x['mimeType'] != "application/vnd.google-apps.folder" 
    ]
  def download_file_name(self, file_name):
      if not any(extension in file_name for extension in GoogleDriveDatabase.FILE_EXTENSIONS()):
        file_name += ".jpg"
      return file_name
  def download_file(self, file, folder:str, **kwargs) -> str:
      check_already_exist = kwargs.get("check_local", False)
      file_name = os.path.join(folder, file['title'])
      file_name = self.download_file_name(file_name)
      if check_already_exist:
        local_files = [os.path.join(folder, file) for file in os.listdir(folder)]
        if file_name in local_files:
          return file_name
      file.GetContentFile(file_name)
      print("downloaded", file_name)
      return file_name

  def download(self, character:str,folder:str):
    file_list = self.getFiles(character)
    os.makedirs(folder, exist_ok=True)
    returnlist = []
    
    for file in file_list:
      returnlist.append(self.download_file(file, folder, check_local=True))


    return tuple(returnlist)

  def checkFolder(self, name):
    assert isinstance(name, str)
    return name in self.folders.keys()

auth.authenticate_user() # Google auth stuff, make sure to sign in with your ucsb account
gauth = GoogleAuth() # Google auth stuff
gauth.credentials = GoogleCredentials.get_application_default() # Google auth stuff
drive = GoogleDrive(gauth) # Google auth stuff

In [0]:
#Openpose block, Run this before making any calls to Openpose
import os
import json
from os.path import exists, join, basename, splitext

def zipOpenpose():
  if not os.path.isfile("openpose"):
    !zip -r openpose.zip openpose
def unzipOpenpose():
  !unzip -o openpose.zip
  !clear
  print("Openpose installed!")

def getOpenpose():
  git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
  project_name = splitext(basename(git_repo_url))[0]
  OPENPOSE_DATABASE = "1LE7a-eAJ5zwtmht4_ItT5bIzmnikNK5Q"
  openpose_database = GoogleDriveDatabase(drive,OPENPOSE_DATABASE)
  storage_reference = openpose_database.getFiles("Storage")
  if not exists(project_name):
    # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
    # install new CMake becaue of CUDA10
    !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
    !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
    # clone openpose
    !git clone -q --depth 1 $git_repo_url
    !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
    # install system dependencies
    !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
    # build openpose
    !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  #   if len(storage_reference) == 1:
  #     print("Will download already compiled openpose")
  #     downloaded = openpose_database.download_file(storage_reference[0], "")
  #     unzipOpenpose()
  #     return
  #   elif len(storage_reference) > 1:
  #     print("More than 1 storage reference found! Will delete them and reupload")
  #     [x.Delete() for x in storage_reference]

  #   !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

  #   zipOpenpose()

  #   openpose_database.upload("openpose.zip", "Storage", "zip")
  # else:
  #   if len(storage_reference) == 0:
  #     print("Will upload openpose")
  #     zipOpenpose()
  #     openpose_database.upload("openpose.zip", "Storage", "zip") 

getOpenpose()

1 folders loaded
Selecting previously unselected package libgflags2.2.
(Reading database ... 145113 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Prepar

In [0]:
import os
import json
import subprocess

def imageToJson(image_directory, json_directory):
  os.makedirs(image_directory, exist_ok=True)
  os.makedirs(json_directory, exist_ok=True)
  !cd openpose && ./build/examples/openpose/openpose.bin --image_dir {image_directory} --hand --write_json {json_directory} --display 0 --render_pose 0

#Run this on any json file generated by Openpose to get a list of the coordinates of the right hand
def getHandCoords(directory) -> list:
  input_file = open (directory)
  json_array = json.load(input_file)
  if len(json_array['people'])!=0 and len(json_array['people'][0])!=0:
    hand = json_array['people'][0]['hand_right_keypoints_2d']
    xAx=hand[::3]
    yAx=hand[1::3]
    hand = []
    for i in range(21):
      hand.append(xAx[i])
      hand.append(yAx[i])
    return hand
  hand = []
  return hand

#Run this over the folder where the results from imageToJson were stored
#Ex. if you ran imageToJson over the folder A, then run folderRun over A_json
def folderRun(directory):
  removed_count = 0
  processed = []
  for file in os.listdir(directory):
    if file.split(".")[-1] == "json":
      new_name = file.replace("_keypoints", "")
      if file != new_name:
        os.rename(os.path.join(directory, file), os.path.join(directory, new_name))
        file = new_name
      full_file_directory = os.path.join(directory, file)
      hand = getHandCoords(full_file_directory)
      # While we normally don't want to include files with bad or no data, it may be best to remove this type of data on the stage of download
      # if (not hand) or all(not x for x in hand):
      #   removed_count += 1
      #   os.remove(full_file_directory)
      #   continue
      # jsonStore = {"image":hand}
      fileStore= open(full_file_directory,"w")
      json.dump(hand,fileStore)
      processed.append(full_file_directory)
  # print("{} files did not contain any hands".format(removed_count))
  print("{} files were processed".format(len(processed)))
  return processed

def Openpose(input_directory, output_directory):
  !rm -rf {output_directory}
  imageToJson(input_directory, output_directory)
  return folderRun(output_directory)

DOWNLOAD_DATABASE = "1RqiJwO6i1KJx54tRC3QJcHWjiooCLJNC"
UPLOAD_DATABASE = "1kLMivfz2q7DFwa57ddy-rCJmsYFTWhMW"
download_database = GoogleDriveDatabase(drive, DOWNLOAD_DATABASE)
upload_database = GoogleDriveDatabase(drive,UPLOAD_DATABASE)
CHARACTERS = ("A", "B", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y")

def download_characters(): # Only downloads characters
  BASE_DIR = "images/"
  results = {}
  for c in CHARACTERS:
    results[c] = download_database.download(c, os.path.join(BASE_DIR, c + "/"))
    print("{} pictures downloaded for {}".format(len(results[c]), c))
  return results
    
def DownloadCharacterAndUploadJson(character):
  strip_name = lambda x: x.split("/")[-1].split(".")[0] # Function to remove extensions (i.e. .json .jpg) from filenames
  character_jsons_original = upload_database.getFiles(character) # All .json files created for this character
  character_jsons_names = [strip_name(x['title']) for x in character_jsons_original] # Name of all .json files already created for this character
  picture_references = download_database.getFiles(character) # All current pictures for this character
  picture_reference_names = [strip_name(x['title']) for x in picture_references] # Name of all pictures
  already_analyzed_names = set() # Names of all already analyzed items
  pictures_to_analyze = [] # Picture that yet do not have a json file
  BASE_FOLDER = "images/" # Directory where all images are
  character_folder = os.path.join(BASE_FOLDER, character + "/") # Folder in which the images for this character are in (i.e. images/A)
  os.makedirs(character_folder, exist_ok=True) # Make the character folder if it doesn't exist
  print("There are {} images for character {}".format(len(picture_references), character))
  print("There are {} json files already made for character {}".format(len(character_jsons_names), character))
  for json_original in character_jsons_original: # For every json file on the google drive...
    if strip_name(json_original['title']) not in picture_reference_names: # ...if the json file is not in the list of pictures...
      print("{} deleted because no image was found to relate to it".format(json_original['title']))
      json_original.Delete() # ...then delete the file

  for picture in picture_references: # For every picture on google drive...
    if strip_name(picture['title']) in character_jsons_names: # ...if that picture already has a json uploaded...
      already_analyzed_names.add(download_database.download_file_name(picture['title'])) # ...add its name to the list of already analyzed pictures
      continue
    else:
      pictures_to_analyze.append(download_database.download_file(picture, character_folder, check_local=True)) # ...else download it
  

  local_duplicates = [os.remove(os.path.join(character_folder, x)) for x in os.listdir(character_folder) if x in already_analyzed_names] # if a picture is in the already analyzed list and exists in the local directory, delete it so that openpose doesn't waste time running it
  print("{} local duplicates deleted".format(len(local_duplicates)))
  if len(os.listdir(character_folder)) == 0:
    print("{} has no images to process".format(character))
    return

  BASE_IMAGES_DIR = "/content/images/"
  BASE_JSON_DIR = "/content/json/"
  json_created = Openpose(os.path.join(BASE_IMAGES_DIR, character), os.path.join(BASE_JSON_DIR, character)) # runs openpose on all images in a given path and outputs each image to an output path
  for json_file in json_created: # For every local json file created...
    upload_database.upload(json_file, character, "json") #...upload it to google drive


for c in CHARACTERS:
  DownloadCharacterAndUploadJson(c)



  

    
# download_characters()


25 folders loaded
24 folders loaded
There are 490 images for character A
There are 515 json files already made for character A
A_0d08b329-e525-4ed3-af11-9072abfd407f.json deleted because no image was found to relate to it
A_0a2260bc-d23a-47d3-aa3c-54b34da91692.json deleted because no image was found to relate to it


ApiRequestError: ignored